<a href="https://colab.research.google.com/github/AgungDanuWijaya/optDFT/blob/main/xcnn(op).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyscf==2.4.0

In [ ]:
pip install numpy==1.26.4

In [ ]:
pip install jax

In [ ]:
pip install torch==2.1.1

In [ ]:
!git clone https://github.com/AgungDanuWijaya/optDFT.git

In [ ]:
from pyscf import gto, dft, scf, cc
from optDFT.test.xcnn_op import ml_xc as ml_xc
mol = gto.Mole()
mol.verbose = 4

mol.atom="""8            .000000     .000000     .119262
1            .000000     .763239    -.477047
1            .000000    -.763239    -.477047"""
mol.charge=0
mol.spin  =0
mol.basis = "aug-cc-pvdz"
mol.build()
mfl = dft.UKS(mol)
mfl.define_xc_(ml_xc.eval_xc_lyp_b, xctype='GGA', hyb=ml_xc.w_[0])
mfl.xc = "HF,"
energy=mfl.kernel()
print(energy)
